In [1]:
import random
from genetic_functions import *

#### LOAD THE DATA

In [ ]:
file_path = "data/jobshop1.txt"  

#Three instances with differences in jobs and machines are selected for testing in different populations
instances_to_extract = ["ft06", "abz5", "abz7"]

instances = extract_instances_to_dict(file_path, instances_to_extract)

# Check
if not instances:
    print("No instances were extracted. Check the names in instances_to_extract and the source file.")
else:
    for instance_name, instance_data in instances.items():
        print(f"instance: {instance_name}")
        print(f"Number of jobs: {instance_data['num_jobs']}")
        print(f"Number of machines: {instance_data['num_machines']}")
        print("Time matrix:")
        for row in instance_data["processing_times"]:
            print(row)
        print("------")

In [4]:
ft06_data = instances["ft06"]
abz5_data = instances["abz5"]

In [5]:
ft06_data

{'num_jobs': 6,
 'num_machines': 6,
 'processing_times': [[2, 1, 0, 3, 1, 6, 3, 7, 5, 3, 4, 6],
  [1, 8, 2, 5, 4, 10, 5, 10, 0, 10, 3, 4],
  [2, 5, 3, 4, 5, 8, 0, 9, 1, 1, 4, 7],
  [1, 5, 0, 5, 2, 5, 3, 3, 4, 8, 5, 9],
  [2, 9, 1, 3, 4, 5, 5, 4, 0, 3, 3, 1],
  [1, 3, 3, 3, 5, 9, 0, 10, 4, 4, 2, 1]]}

### Chromosome encoding

### Validation of the chromosome

### Compute the fitness

In [9]:
# Datos de entrada
num_jobs = ft06_data['num_jobs']
num_machines = ft06_data['num_machines']
processing_times = ft06_data['processing_times']

chromosome = generate_chromosome(num_jobs, num_machines)
print("Cromosoma generado:", chromosome)
print(len(chromosome))

# Calcular el fitness con validación
fitness = compute_fitness_with_validation(chromosome, num_jobs, num_machines, processing_times)
if fitness == float('inf'):
    print("El cromosoma no es válido.")
else:
    print("Makespan (fitness) de la solución:", fitness)


Cromosoma generado: [5, 3, 0, 1, 5, 4, 4, 5, 1, 2, 4, 4, 0, 4, 5, 2, 4, 3, 2, 3, 0, 1, 0, 0, 1, 1, 2, 3, 1, 5, 3, 3, 5, 2, 2, 0]
36
Makespan (fitness) de la solución: 85


### Genetic Operators

#### Turnament selection

In [64]:
num_jobs = ft06_data['num_jobs']
num_machines = ft06_data['num_machines']
processing_times = ft06_data['processing_times']
population = [generate_chromosome(num_jobs, num_machines) for _ in range(100)]
fitness_values = [compute_fitness_with_validation(ind, num_jobs, num_machines, processing_times) for ind in population]

selected_roulette = roulette_selection(population, fitness_values)
print("Cromosoma seleccionado:", selected_roulette)

selected_tournament = tournament_selection(population, fitness_values)
print("Cromosoma seleccionado:", selected_tournament)

Cromosoma seleccionado: [0, 4, 4, 1, 0, 4, 3, 5, 5, 0, 2, 1, 0, 3, 5, 1, 2, 2, 4, 5, 2, 1, 3, 0, 1, 2, 3, 1, 5, 5, 4, 3, 3, 2, 4, 0]
Cromosoma seleccionado: [1, 5, 5, 3, 5, 1, 2, 4, 3, 1, 3, 0, 3, 2, 0, 5, 0, 5, 1, 4, 2, 0, 4, 0, 3, 1, 1, 2, 2, 4, 4, 3, 4, 0, 2, 5]


#### Crossover

In [59]:
# Cromosomas de ejemplo
parent1 = [4, 3, 1, 1, 5, 5, 5, 1, 4, 0, 2, 3, 2, 3, 1, 5, 3, 1, 0, 5, 0, 4, 2, 2, 1, 4, 3, 2, 4, 2, 0, 3, 5, 0, 0, 4]
parent2 = [3, 5, 1, 4, 1, 3, 4, 1, 0, 2, 5, 4, 2, 5, 3, 4, 2, 0, 5, 0, 2, 4, 0, 1, 3, 3, 2, 3, 0, 5, 1, 1, 2, 5, 0, 4]

# Crossover de un punto
child_one_point = one_point_crossover(parent1, parent2)
print("Hijo (crossover de un punto):", child_one_point)

# Crossover de dos puntos
child = cycle_crossover(parent1, parent2)
print(child)


Hijo (crossover de un punto): [4, 3, 1, 1, 5, 5, 5, 1, 0, 2, 5, 4, 2, 5, 3, 4, 2, 0, 5, 0, 2, 4, 0, 1, 3, 3, 2, 3, 0, 5, 1, 1, 2, 5, 0, 4]
[4, 3, 1, 1, 5, 5, 5, 1, 4, 0, 2, 3, 2, 3, 1, 5, 3, 1, 0, 5, 0, 4, 2, 2, 1, 4, 3, 2, 4, 2, 0, 3, 5, 0, 0, 4]


#### Mutation

In [48]:
# Cromosoma de ejemplo
cromosoma = [1, 2, 3, 4, 5, 6, 7, 8]

# Mutación por rotación
cromo = mutation(cromosoma)
cromosoma_mutado = mutate_rotation(cromosoma.copy())
print("Cromosoma después de mutación por rotación:", cromosoma_mutado, cromo)


Cromosoma después de mutación por rotación: [2, 6, 7, 8, 1, 5, 3, 4] [1, 5, 3, 4, 2, 6, 7, 8]


# Genetic algorithm

In [44]:
def genetic_algorithm(data, population_size=20, crossover_prob=0.8, mutation_prob=0.2, max_generations=100, no_improve_limit=10):
    num_jobs = data['num_jobs']
    num_machines = data['num_machines']
    processing_times = data['processing_times']

    # Inicializar población aleatoria
    population = [generate_chromosome(num_jobs, num_machines) for _ in range(population_size)]

    print(len(population))

    # Evaluar la población inicial
    fitness_values = [compute_fitness_with_validation(ind, num_jobs, num_machines, processing_times) for ind in population]
    print(len(fitness_values))
    best_fitness = min(fitness_values)
    best_solution = population[fitness_values.index(best_fitness)]
    print(best_solution)
    no_improve_count = 0

    for generation in range(max_generations):
        new_population = []

        # Generar nueva población
        while len(new_population) < population_size:
            # Selección de padres
            parent1 = tournament_selection(population, fitness_values)
            parent2 = tournament_selection(population, fitness_values)

            # Cruce
            if random.random() < crossover_prob:
                child = crossover(parent1, parent2, num_jobs)
                print(child)
            else:
                child = parent1

            # Mutación
            if random.random() < mutation_prob:
                child = mutation(child)

            new_population.append(child)

        # Evaluar la nueva población
        population = new_population
        fitness_values = [compute_fitness_with_validation(ind, num_jobs, num_machines, processing_times) for ind in population]

        # Actualizar mejor solución
        current_best_fitness = min(fitness_values)
        if current_best_fitness < best_fitness:
            best_fitness = current_best_fitness
            best_solution = population[fitness_values.index(current_best_fitness)]
            no_improve_count = 0
        else:
            no_improve_count += 1

        # Criterio de convergencia
        if no_improve_count >= no_improve_limit:
            print(f"Convergencia alcanzada después de {generation + 1} generaciones.")
            break

        # Imprimir progreso
        print(f"Generación {generation + 1}: Mejor fitness = {best_fitness}")

    return best_solution, best_fitness

In [45]:
best_solution, best_fitness = genetic_algorithm(ft06_data, population_size=50, crossover_prob=0.9, mutation_prob=0.1)
print("Mejor solución encontrada:", best_solution)
print("Makespan (fitness) de la mejor solución:", best_fitness)

50
50
[2, 4, 3, 3, 5, 4, 1, 5, 5, 3, 0, 0, 2, 1, 0, 4, 3, 2, 1, 5, 3, 0, 2, 4, 4, 4, 5, 2, 1, 0, 1, 2, 1, 3, 0, 5]


IndexError: list index out of range